In [7]:
from aiida import load_profile, orm
load_profile()

Profile<uuid='5ca88b1d2d41436bb1747cc5c08179db' name='default'>

In [8]:
from aiida_worktree import node

# define add node
@node.calcfunction()
def add(x, y):
    return x + y

# define multiply node
@node.calcfunction()
def multiply(x, y):
    return x*y

In [9]:
from aiida_worktree import WorkTree
from aiida import orm

wt = WorkTree("first_workflow")
wt.nodes.new(add, name="add1", x=orm.Int(2), y=orm.Int(3))
wt.nodes.new(add, name="add2", x=orm.Int(2))
wt.links.new(wt.nodes["add1"].outputs[0], wt.nodes["add2"].inputs["y"])


In [10]:
wt.submit(wait=True)

WorkTree node created, PK: 939


In [11]:
print("State of WorkTree:   {}".format(wt.state))
print('Result of add      : {}'.format(wt.nodes["add1"].node.outputs.result.value))
print('Result of multiply : {}'.format(wt.nodes["add2"].node.outputs.result.value))

State of WorkTree:   FINISHED
Result of add      : 5
Result of multiply : 7


## Basics: scf+kc conversion.

In [12]:
from aiida_worktree import build_node
ndata = {"path": "aiida_koopmans.calculations.kcw.KcwCalculation"}
kcw_calcjob = build_node(ndata)

ndata = {"path": "aiida_quantumespresso.workflows.pw.base.PwBaseWorkChain"}
pw_baseWchain = build_node(ndata)

In [13]:
node = kcw_calcjob()
print("Inputs:")
for input in node.inputs:
    if "." not in input.name:
        print(f"  - {input.name}")
print("Outputs:")
for output in node.outputs:
    if "." not in output.name:
        print(f"  - {output.name}")

Inputs:
  - metadata
  - code
  - monitors
  - remote_folder
  - parameters
  - settings
  - parent_folder
Outputs:
  - remote_folder
  - remote_stash
  - retrieved
  - output_parameters
  - bands


In [14]:
node = pw_baseWchain()
print("Inputs:")
for input in node.inputs:
    if "." not in input.name:
        print(f"  - {input.name}")
print("Outputs:")
for output in node.outputs:
    if "." not in output.name:
        print(f"  - {output.name}")

Inputs:
  - metadata
  - max_iterations
  - clean_workdir
  - handler_overrides
  - pw
  - kpoints
  - kpoints_distance
  - kpoints_force_parity
Outputs:
  - remote_folder
  - remote_stash
  - retrieved
  - output_parameters
  - output_structure
  - output_trajectory
  - output_band
  - output_kpoints
  - output_atomic_occupations


In [83]:
from ase.io import read

ozone = read("/home/jovyan/work/koopmans_calcs/tutorial_1/ozon.xsf")
ozone.cell = [[14.1738, 0.0, 0.0],
                  [0.0, 12.0, 0.0],
                  [0.0, 0.0, 12.66]]
structure = orm.StructureData(ase=ozone)

pw_code = orm.load_code("pw-7.2@localhost")
kcw_code = orm.load_code("kcw-7.2@localhost")
pseudo_family = orm.load_group("sg15_v1.2/pbe/SR")
pseudos = pseudo_family.get_pseudos(structure=structure) 


scf_params = {'CONTROL': {'calculation': 'scf',
  'forc_conv_thr': 0.0001,
  'tprnfor': True,
  'tstress': True,
  'verbosity': 'high',
  'etot_conv_thr': 3.0000000000000004e-05},
 'SYSTEM': {'nosym': False,
  'occupations': 'fixed',
  'nbnd': 10,
  'tot_magnetization': 0,
  'nspin': 2,
  'starting_magnetization(1)': 0.0,
  'assume_isolated': 'mt',
  'ecutwfc': 50.0,
  'ecutrho': 200.0},
 'ELECTRONS': {'electron_maxstep': 80, 'mixing_beta': 0.4, 'conv_thr': 6e-10}}

wann2kcw_params = {
        "CONTROL":{
                'kcw_iverbosity':1,
                'kcw_at_ks'      :True,
                'calculation'    :'wann2kcw',
                'lrpa'           :False,
                'mp1'            :1,
                'mp2'            :1,
                'mp3'            :1,
                'homo_only'      :False,
                'read_unitary_matrix' : False,
                'l_vcut'         :False,
                'assume_isolated':'m-t',
                'spin_component' :1,
                },
        "WANNIER":{
                "check_ks"       : True,
                "num_wann_occ"   : 9,
                "num_wann_emp"   : 1,
                "have_empty"     : True,
                "has_disentangle": False,
                    },
    }

screening_dict = {
    'tr2'         : 1e-18,
    'nmix'        : 4,
    'niter'       : 33,
    'check_spread': True,
}

kc_screen_params = {
        "CONTROL":{
                'kcw_iverbosity':1,
                'kcw_at_ks'      :True,
                'calculation'    :'screen',
                'lrpa'           :False,
                'mp1'            :1,
                'mp2'            :1,
                'mp3'            :1,
                'homo_only'      :False,
                'read_unitary_matrix' : False,
                'l_vcut'         :False,
                'assume_isolated':'m-t',
                'spin_component' :1,
                },
        "SCREEN":screening_dict,
        "WANNIER":{
                "check_ks"       : True,
                "num_wann_occ"   : 9,
                "num_wann_emp"   : 1,
                "have_empty"     : True,
                "has_disentangle": False,
                    },
    }

ham_dict = {
   'do_bands'       : False,
   'use_ws_distance': True,
   'write_hr'       : True,
   'l_alpha_corr'   : False,
}

kc_ham_params = {
        "CONTROL":{
                'kcw_iverbosity':1,
                'kcw_at_ks'      :True,
                'calculation'    :'ham',
                'lrpa'           :False,
                'mp1'            :1,
                'mp2'            :1,
                'mp3'            :1,
                'homo_only'      :False,
                'read_unitary_matrix' : False,
                'l_vcut'         :False,
                'assume_isolated':'m-t',
                'spin_component' :1,
                },
        "HAM":ham_dict,
        "WANNIER":{
                "check_ks"       : True,
                "num_wann_occ"   : 9,
                "num_wann_emp"   : 1,
                "have_empty"     : True,
                "has_disentangle": False,
                    },
    }


metadata = {
    'options': {
        'max_wallclock_seconds': 3600,
        'resources': {
            "num_machines": 1,
            "num_mpiprocs_per_machine": 10,
            "num_cores_per_mpiproc": 1,
        },
        'custom_scheduler_commands': u"export OMP_NUM_THREADS="+str(1),
    }
}

In [84]:
from aiida_worktree import WorkTree

wt = WorkTree("scf_kcw_conversion")
scf = wt.nodes.new(pw_baseWchain, name="scf")
scf.set_from_protocol(
    code = pw_code,
    structure = structure,
    overrides = {"pw":{"parameters":scf_params}}
)
scf.inputs["pw"].value["metadata"]=metadata

In [85]:
scf.inputs["pw"].value["metadata"]=metadata
scf.inputs["pw"].value["pseudos"]=pseudos

In [86]:
kc_conv = wt.nodes.new(kcw_calcjob, name="kc_conversion")
kc_conv.set(
    {
        "code": kcw_code,
        "parameters": orm.Dict(wann2kcw_params),
        "metadata": metadata,
    }
)

In [87]:
kc_screen = wt.nodes.new(kcw_calcjob, name="kc_screen")
kc_screen.set(
    {
        "code": kcw_code,
        "parameters": orm.Dict(kc_screen_params),
        "metadata": metadata,
    }
)

In [88]:
kc_ham = wt.nodes.new(kcw_calcjob, name="kc_ham")
kc_ham.set(
    {
        "code": kcw_code,
        "parameters": orm.Dict(kc_ham_params),
        "metadata": metadata,
    }
)

In [89]:
wt.links.new(scf.outputs["remote_folder"], kc_conv.inputs["parent_folder"])
wt.links.new(kc_conv.outputs["remote_folder"], kc_screen.inputs["parent_folder"])
wt.links.new(kc_screen.outputs["remote_folder"], kc_ham.inputs["parent_folder"])

In [90]:
wt.submit(wait=True)

WorkTree node created, PK: 1035


In [91]:
pwd

'/home/jovyan/codes/aiida-koopmans'